In [1]:
import pandas as pd

csv_filename = "./dataset_alarms_anonymized.csv"
dataset = pd.read_csv(csv_filename)
dataset

,alarm_0,alarm_1,alarm_2,alarm_3,alarm_4,alarm_5,alarm_6,alarm_7,alarm_8,alarm_9,...,alarm_155,alarm_156,alarm_157,alarm_158,alarm_159,alarm_160,alarm_161,alarm_162,alarm_163,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,28,0,0,3
1665,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3
1666,0,0,0,0,0,0,0,0,0,0,...,24,0,0,0,0,0,0,0,0,0
1667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [2]:
!pip install sdv nbformat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Usin

In [3]:
import sdv

print(sdv.version.public)

1.13.1


In [4]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.sampling import Condition
from typing import Optional


def balance_gan(
    data,
    label_column="label",
    random_state=42,
    fixed_number_of_samples: Optional[int] = None,
):
    # Split data by class
    data = data.sample(frac=1, random_state=random_state).reset_index(drop=True)
    class_data = data.groupby(label_column)
    for cls, subset in class_data:
        print(f"Class {cls} has {len(subset)} samples")

    # Determine target class size (size of the largest class)
    if not fixed_number_of_samples:
        target_size = class_data.size().max()
        print(f"Majority class size: {target_size}")

    # Generate synthetic data for minority classes
    synthetic_data = pd.DataFrame()
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)

    # Initialize and fit the CTGANSynthesizer
    ctgan = CTGANSynthesizer(metadata=metadata, epochs=600)
    ctgan.fit(data)

    # Calculate the number of samples needed for each class
    if fixed_number_of_samples:
        samples_needed = {cls: fixed_number_of_samples for cls, _ in class_data}
    else:
        samples_needed = {cls: target_size - len(subset) for cls, subset in class_data}
        print(f"Samples needed to evenly balance: {samples_needed}")

    # Generate conditions for each class to do conditional sampling
    conditions = [
        Condition(column_values={label_column: cls}, num_rows=samples_needed[cls])
        for cls in samples_needed
    ]

    # Sample synthetic data for each class
    synthetic_data = ctgan.sample_from_conditions(conditions=conditions)

    # Combine original and synthetic data
    balanced_data = pd.concat([data, synthetic_data], ignore_index=True)
    return ctgan, balanced_data, synthetic_data, metadata

In [1]:
ctgan, evenly_balanced_dataset, evenly_synthentic_data, evenly_metadata = balance_gan(
    dataset
)
ctgan2, fixed_add_dataset, fixed_synthetic_data, fix_metadata = balance_gan(
    dataset, fixed_number_of_samples=300
)

NameError: name 'balance_gan' is not defined

In [6]:
fig = ctgan.get_loss_values_plot()
fig.show()

In [7]:
fig = ctgan2.get_loss_values_plot()
fig.show()

In [8]:
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality
from sdv.evaluation.single_table import get_column_plot

# 1. perform basic validity checks
diagnostic = run_diagnostic(dataset, evenly_synthentic_data, evenly_metadata)

# 2. measure the statistical similarity
quality_report = evaluate_quality(dataset, evenly_synthentic_data, evenly_metadata)

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 165/165 [00:00<00:00, 1867.76it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 203.61it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 165/165 [00:00<00:00, 353.00it/s]|
Column Shapes Score: 85.79%

(2/2) Evaluating Column Pair Trends: |██████████| 13530/13530 [04:35<00:00, 49.09it/s]|
Column Pair Trends Score: 95.05%

Overall Score (Average): 90.42%



In [9]:
evenly_balanced_dataset.to_csv("evenly_balanced_dataset.csv", index=False)
fixed_add_dataset.to_csv("fixed_add_dataset.csv", index=False)

In [10]:
# group by label in the original dataset
grouped_original = dataset.groupby("label")
print("Original Dataset:")
print(grouped_original.size())

# group by label in the balanced dataset
grouped_balanced = evenly_balanced_dataset.groupby("label")
print("Balanced Dataset:")
print(grouped_balanced.size())

# group by label in the balanced dataset
grouped_fixed_add = fixed_add_dataset.groupby("label")
print("Fixed Add Dataset:")
print(grouped_fixed_add.size())

Original Dataset:
label
0    515
1    611
2    207
3    336
dtype: int64
Balanced Dataset:
label
0    611
1    611
2    611
3    611
dtype: int64
Fixed Add Dataset:
label
0    815
1    911
2    507
3    636
dtype: int64
